In [0]:
# Load transactional data
transaction_df = spark.read.csv("/mnt/blob/transactional_data.csv", header=True, inferSchema=True)

# Load customer data
customer_df = spark.read.csv("/mnt/blob/customer_data.csv", header=True, inferSchema=True)

# Load product data
product_df = spark.read.csv("/mnt/blob/product_data.csv", header=True, inferSchema=True)

# Load reviews data
reviews_df = spark.read.csv("/mnt/blob/reviews_data.csv", header=True, inferSchema=True)

# Load inventory data
inventory_df = spark.read.csv("/mnt/blob/inventory_data.csv", header=True, inferSchema=True)



In [0]:

from pyspark.sql.functions import col

customer_df = customer_df.withColumn("customer_id", col("customer_id").cast("int")) \
                         .withColumn("signup_date", col("signup_date").cast("date")) \
                         .withColumn("first_name", col("first_name").cast("string")) \
                         .withColumn("last_name", col("last_name").cast("string")) \
                         .withColumn("email", col("email").cast("string")) \
                         .withColumn("address", col("address").cast("string"))


In [0]:
product_df = product_df.withColumn("product_id", col("product_id").cast("int")) \
                       .withColumn("price", col("price").cast("decimal(10, 2)")) \
                       .withColumn("stock_quantity", col("stock_quantity").cast("int")) \
                       .withColumn("product_name", col("product_name").cast("string")) \
                       .withColumn("category", col("category").cast("string"))


In [0]:
reviews_df = reviews_df.withColumn("review_id", col("review_id").cast("int")) \
                       .withColumn("customer_id", col("customer_id").cast("int")) \
                       .withColumn("product_id", col("product_id").cast("int")) \
                       .withColumn("rating", col("rating").cast("int")) \
                       .withColumn("review_date", col("review_date").cast("date")) \
                       .withColumn("review_text", col("review_text").cast("string"))


In [0]:
inventory_df = inventory_df.withColumn("product_id", col("product_id").cast("int")) \
                           .withColumn("current_stock", col("current_stock").cast("int")) \
                           .withColumn("reorder_level", col("reorder_level").cast("int"))


In [0]:
transaction_df = transaction_df.withColumn("transaction_id", col("transaction_id").cast("int")) \
                               .withColumn("customer_id", col("customer_id").cast("int")) \
                               .withColumn("transaction_date", col("transaction_date").cast("timestamp")) \
                               .withColumn("transaction_amount", col("transaction_amount").cast("decimal(10, 2)")) \
                               .withColumn("product_id", col("product_id").cast("int")) \
                               .withColumn("quantity", col("quantity").cast("int")) \
                               .withColumn("payment_type", col("payment_type").cast("string"))


In [0]:
# Verify the schema after casting
transaction_df.printSchema()
customer_df.printSchema()
product_df.printSchema()
reviews_df.printSchema()
inventory_df.printSchema()


In [0]:
from pyspark.sql.functions import when, avg, col

# Handle null transaction_amount (set to 0)
transaction_df = transaction_df.withColumn(
    "transaction_amount", 
    when(col("transaction_amount").isNull(), 0).otherwise(col("transaction_amount"))
)

transaction_df = transaction_df.fillna("No value provided", subset=["payment_type", "product_id"])
customer_df = customer_df.fillna("No value provided", subset=["first_name", "last_name", "email", "address"])
product_df = product_df.fillna("No value provided", subset=["product_name", "category"])

# Calculate average rating
avg_rating = reviews_df.select(avg(col("rating"))).first()[0]
avg_rating
# Fill missing rating with the average rating
reviews_df = reviews_df.withColumn("rating", when(col("rating").isNull(), avg_rating).otherwise(col("rating")))

reviews_df = reviews_df.fillna("No feedback provided", subset=["review_text"])
avg_rating



Out[6]: 4.2105263157894735

In [0]:
from azure.storage.blob import BlobServiceClient

# Split the SAS URL into the base URL and the SAS token
account_url = "https://capstorage11.blob.core.windows.net"
sas_token = "sp=rac&st=2024-10-14T18:52:48Z&se=2024-10-26T02:52:48Z&sv=2022-11-02&sr=c&sig=1BPu0NlSZVLMGGPhNRa6xIdZLqIYrOSwbMayuDFIc%2B0%3D"

# Initialize the BlobServiceClient with the base URL and the SAS token
blob_service_client = BlobServiceClient(account_url=account_url, credential=sas_token)
container_client = blob_service_client.get_container_client("initialdata")

In [0]:
# Load the respective CSV files into DataFrames
transaction_df = spark.read.csv("/mnt/blob/transactional_data.csv", header=True, inferSchema=True)
customer_df = spark.read.csv("/mnt/blob/customer_data.csv", header=True, inferSchema=True)
product_df = spark.read.csv("/mnt/blob/product_data.csv", header=True, inferSchema=True)
reviews_df = spark.read.csv("/mnt/blob/reviews_data.csv", header=True, inferSchema=True)
inventory_df = spark.read.csv("/mnt/blob/inventory_data.csv", header=True, inferSchema=True)

# JDBC connection properties
jdbc_url = "jdbc:sqlserver://capstoneservo.database.windows.net:1433;database=capstonedb"
connection_properties = {
    "user": "ash@capstoneservo",
    "password": "NOTmaurya69",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Append data to the respective SQL tables without dropping the tables
transaction_df.write.jdbc(url=jdbc_url, table="cap.Transactions", mode="append", properties=connection_properties)
#customer_df.write.jdbc(url=jdbc_url, table="cap.Customers", mode="append", properties=connection_properties)
#product_df.write.jdbc(url=jdbc_url, table="cap.Products", mode="append", properties=connection_properties)
reviews_df.write.jdbc(url=jdbc_url, table="cap.Reviews", mode="append", properties=connection_properties)
inventory_df.write.jdbc(url=jdbc_url, table="cap.Inventory", mode="append", properties=connection_properties)


In [0]:
# JDBC connection properties
jdbc_url = "jdbc:sqlserver://capstoneservo.database.windows.net:1433;database=capstonedb"
connection_properties = {
    "user": "ash@capstoneservo",
    "password": "NOTmaurya69",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
transaction_df = spark.read.csv("/mnt/blob/transactional_data.csv", header=True, inferSchema=True)
transaction_df.write.jdbc(url=jdbc_url, table="cap.Transactions", mode="append", properties=connection_properties)
